https://towardsdatascience.com/genetic-algorithms-in-python-using-the-deap-library-e67f7ce4024c

In [70]:
import pandas as pd
import numpy as np
import random
from deap import base
from deap import creator
from deap import tools

In [71]:
# goal percentages
total_calories = 2500 * 7
percentage_prot = 0.3
percentage_carb = 0.5
percentage_fat = 0.2
# compute total calories per macro
cal_prot = round(percentage_prot * total_calories)
cal_carb = round(percentage_carb * total_calories)
cal_fat = round(percentage_fat * total_calories)
print(cal_prot, cal_carb, cal_fat)
# fixed info on macro nutrients: calories per gram of protein, carb and fat
prot_cal_p_gram = 4
carb_cal_p_gram = 4
fat_cal_p_gram = 9
# goal grams
gram_prot = cal_prot / prot_cal_p_gram
gram_carb = cal_carb / carb_cal_p_gram
gram_fat = cal_fat / fat_cal_p_gram
print(gram_prot, gram_carb, gram_fat)

5250 8750 3500
1312.5 2187.5 388.8888888888889


In [72]:
# per week: min, max, cal unit, prot g,  fat g, carb g
products_table = pd.DataFrame.from_records([
    ['Banana 1u', 0, 4, 89, 1, 0, 23],
    ['Mandarin 1u', 0, 4, 40, 1, 0, 10],
    ['Ananas 100g', 0, 7, 50, 1, 0, 13],
    ['Grapes 100g', 0, 7, 76, 1, 0, 17],
    ['Chocolate 1 bar', 0, 4, 230, 3, 13, 25],
    
    ['Hard Cheese 100g', 0, 8, 350, 28, 26, 2],
    ['Soft Cheese 100g', 0, 8, 374, 18, 33, 1],
    ['Pesto 100g', 0, 8, 303, 3, 30, 4],
    ['Hoummous 100g', 0, 8, 306, 7, 25, 11],
    ['Aubergine Paste 100g', 0, 4, 228, 1, 20, 8],
    
    ['Protein Shake', 0, 5, 160, 30, 3, 5],
    ['Veggie Burger 1', 0, 5, 220, 21, 12, 3],
    ['Veggie Burger 2', 0, 12, 165, 16, 9, 2],
    ['Boiled Egg', 0, 8, 155, 13, 11, 1],
    ['Backed Egg', 0, 16, 196, 14, 15, 1],
    
    ['Baguette Bread Half', 0, 3, 274, 10, 0, 52],
    ['Square Bread 1 slice', 0, 3, 97, 3, 1, 17],
    ['Cheese Pizza 1u', 0, 3, 903, 36, 47, 81],
    ['Veggie Pizza 1u', 0, 3, 766, 26, 35, 85],
    
    ['Soy Milk 200ml', 0, 1, 115, 8, 4, 11],
    ['Soy Chocolate Milk 250ml', 0, 3, 160, 7, 6,20],
    
])
products_table.columns = ['Name', 'Min', 'Max', 'Calories', 'Gram_Prot', 'Gram_Fat', 'Gram_Carb']

products_table

,Name,Min,Max,Calories,Gram_Prot,Gram_Fat,Gram_Carb
0,Banana 1u,0,4,89,1,0,23
1,Mandarin 1u,0,4,40,1,0,10
2,Ananas 100g,0,7,50,1,0,13
3,Grapes 100g,0,7,76,1,0,17
4,Chocolate 1 bar,0,4,230,3,13,25
5,Hard Cheese 100g,0,8,350,28,26,2
6,Soft Cheese 100g,0,8,374,18,33,1
7,Pesto 100g,0,8,303,3,30,4
8,Hoummous 100g,0,8,306,7,25,11
9,Aubergine Paste 100g,0,4,228,1,20,8


# Optimize The Shopping List Univariately To Match Calories

In [73]:
# extract the information of products in a format that is easier to use in the deap algorithms cost function
cal_data = products_table[['Gram_Prot', 'Gram_Fat', 'Gram_Carb']]

prot_data = list(cal_data['Gram_Prot'])
fat_data = list(cal_data['Gram_Fat'])
carb_data = list(cal_data['Gram_Carb'])

In [74]:
# the random initialization of the genetic algorithm is done here
# it gives a list of integers with for each products the number of times it is bought
def n_per_product():
    return random.choices(range(0, 10), k=21)


# this is the function used by the algorithm for evaluation
# I chose it to be the absolute difference of the number of calories in the planning and the goal of calories


def evaluate(individual):
    individual = individual[0]
    tot_prot = sum(x * y for x, y in zip(prot_data, individual))
    tot_fat = sum(x * y for x, y in zip(fat_data, individual))
    tot_carb = sum(x * y for x, y in zip(carb_data, individual))
    cals = prot_cal_p_gram * tot_prot + carb_cal_p_gram * \
        tot_carb + fat_cal_p_gram * tot_fat
    return abs(cals - total_calories),

In [75]:
# this is the setup of the deap library: registering the different function into the toolbox
creator.create("FitnessMin", base.Fitness, weights=(-1.0, ))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("n_per_product", n_per_product)

toolbox.register("individual",
                 tools.initRepeat,
                 creator.Individual,
                 toolbox.n_per_product,
                 n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

C:\Users\AW639XJ\AppData\Roaming\Python\Python37\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\AW639XJ\AppData\Roaming\Python\Python37\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [76]:
# as an example, this is what a population of 10 shopping lists looks like
toolbox.population(n=10)

[[[7, 8, 7, 8, 4, 2, 3, 6, 3, 0, 5, 0, 4, 1, 1, 8, 2, 6, 1, 3, 4]],
 [[6, 3, 5, 6, 2, 3, 9, 0, 3, 1, 5, 0, 5, 3, 5, 8, 7, 5, 9, 1, 4]],
 [[3, 0, 6, 6, 3, 5, 2, 8, 6, 8, 9, 3, 1, 0, 0, 0, 6, 2, 7, 5, 4]],
 [[3, 6, 1, 8, 5, 1, 4, 3, 7, 7, 2, 0, 3, 9, 6, 3, 9, 3, 2, 8, 3]],
 [[7, 0, 3, 2, 0, 0, 1, 6, 0, 6, 0, 2, 1, 9, 1, 5, 0, 9, 1, 4, 3]],
 [[9, 1, 3, 6, 5, 4, 0, 8, 8, 1, 8, 1, 3, 1, 9, 8, 3, 4, 2, 5, 3]],
 [[3, 0, 4, 2, 3, 2, 8, 3, 7, 5, 2, 7, 5, 0, 5, 8, 1, 8, 8, 2, 1]],
 [[3, 2, 8, 8, 8, 2, 5, 5, 0, 1, 6, 4, 9, 2, 8, 7, 3, 7, 5, 9, 5]],
 [[3, 0, 5, 7, 7, 9, 0, 6, 5, 9, 0, 8, 6, 5, 9, 7, 0, 6, 7, 3, 9]],
 [[0, 7, 3, 1, 1, 7, 7, 4, 9, 0, 9, 9, 1, 4, 0, 1, 6, 3, 9, 0, 0]]]

In [77]:
# this is the definition of the total genetic algorithm is executed, it is almost literally copied from the deap library
def main():
    pop = toolbox.population(n=300)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]
    
    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while g < 5000:
        # A new generation
        g = g + 1
        #print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values
            
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print(min(fits), max(fits), mean, std)
    
    best = pop[np.argmin([toolbox.evaluate(x) for x in pop])]
    return best

In [78]:
!python -c "import deap; print(deap.__version__)"

1.3


In [79]:
best_solution = main()
products_table['univariate_choice'] = pd.Series(best_solution[0])
products_table.head()

5.0 15284.0 2982.9266666666667 2335.4295767493304
5.0 8199.0 1601.3666666666666 1397.0299634900066
5.0 8609.0 1316.5033333333333 1399.3626537304601
5.0 6175.0 1131.2033333333334 1200.3567450230046
5.0 5805.0 870.99 1036.9693035797475
2.0 7947.0 1044.2666666666667 1408.5674054000951
2.0 7372.0 1001.6033333333334 1347.6407357510218
2.0 7005.0 849.4666666666667 1209.164902824351
2.0 4971.0 733.83 1017.9536537092442
2.0 6881.0 793.4966666666667 1170.4382327383003
2.0 8132.0 773.9233333333333 1246.1750749616026
2.0 6073.0 615.9566666666667 1090.1925708128613
2.0 4308.0 458.53333333333336 824.2491060892264
2.0 5828.0 434.58666666666664 887.8286673051782
2.0 4363.0 671.2766666666666 1077.5622271848413
2.0 5110.0 576.2666666666667 1028.1904244296816
2.0 6052.0 638.5 1185.1224620266041
2.0 4658.0 307.97 757.132772878487
2.0 6210.0 211.34666666666666 690.3572552108236
2.0 3917.0 126.68666666666667 443.810013957424
2.0 5820.0 133.13333333333333 498.4373269418021
2.0 4394.0 187.01333333333332 624.

2.0 3302.0 147.2 485.3284729610109
2.0 3966.0 150.12666666666667 510.61887021752557
2.0 3038.0 114.06333333333333 415.5747979071343
2.0 3491.0 150.0 553.2187270872164
2.0 6906.0 225.91 800.4952437293637
2.0 3929.0 198.52666666666667 621.2350300990939
2.0 4454.0 215.94666666666666 639.3932466191852
2.0 4097.0 173.23 560.40093721668
2.0 6935.0 176.85 652.4170400646099
2.0 6906.0 188.95333333333335 662.7936364275753
2.0 4172.0 261.63 732.1200947249023
2.0 3254.0 163.57333333333332 509.92516243943965
2.0 4530.0 148.55666666666667 550.1104981021741
2.0 4622.0 144.48666666666668 546.583122518636
2.0 4746.0 236.73 726.6223529225251
2.0 5260.0 131.9 525.2446065088785
2.0 4621.0 186.73 633.5710039924492
2.0 5414.0 241.67666666666668 733.8591863944714
2.0 3269.0 169.55333333333334 563.3238563699886
2.0 3870.0 183.48666666666668 575.112930784516
2.0 5022.0 222.79666666666665 710.223773648528
2.0 3350.0 186.45333333333335 550.5734172862165
2.0 5204.0 181.14333333333335 617.1638756890713
2.0 3372.0

2.0 2555.0 122.81 382.3714954944558
2.0 3929.0 222.58666666666667 667.705610646555
2.0 4013.0 148.2 558.6925749760179
2.0 5582.0 176.57666666666665 592.1381855520626
2.0 7054.0 246.89333333333335 799.9238892683617
2.0 3972.0 178.06333333333333 531.9423082649304
2.0 4921.0 180.16666666666666 621.3415369630101
2.0 4172.0 216.88333333333333 625.6915664997323
2.0 5423.0 238.79666666666665 712.2646245056834
2.0 4592.0 183.92 590.0774132264341
2.0 4725.0 185.70666666666668 639.8911683160574
2.0 3634.0 148.96333333333334 493.4015491012929
2.0 5350.0 211.38333333333333 699.7886750457043
2.0 4926.0 160.86666666666667 607.9291534015749
2.0 5183.0 156.57666666666665 556.6055492496957
2.0 4746.0 173.17666666666668 640.1245858858692
2.0 3533.0 145.54666666666665 518.8613313357969
2.0 4022.0 143.98666666666668 534.8792759326373
2.0 4193.0 152.13 561.3565353379852
2.0 6830.0 190.85666666666665 675.378611932267
2.0 4750.0 131.21666666666667 535.3567997658716
2.0 4686.0 182.86333333333334 603.792148002

2.0 3353.0 165.18 512.1380812502296
2.0 7122.0 232.92666666666668 686.2054512235303
2.0 4030.0 191.21333333333334 592.2946911424714
2.0 8322.0 192.37333333333333 754.334974633654
2.0 4495.0 129.93 504.27332380366903
2.0 5050.0 212.99 683.1191330214665
2.0 4198.0 197.45333333333335 670.4458724030019
2.0 3603.0 205.27333333333334 659.7403973954873
2.0 4518.0 188.38333333333333 622.2507771969612
2.0 4503.0 189.39666666666668 614.3346151098945
2.0 4188.0 197.73333333333332 688.7372664682972
2.0 4086.0 102.22 409.6761626781166
2.0 4756.0 175.91666666666666 596.7656126058947
2.0 3134.0 130.83333333333334 397.30059009380903
2.0 6406.0 168.11333333333334 733.2250044533093
2.0 4019.0 164.25 569.6757330563883
2.0 5437.0 257.89 798.3452832160614
2.0 6749.0 208.86333333333334 702.8794382079728
2.0 4750.0 180.97 648.4060372174214
2.0 5414.0 181.99666666666667 601.0995341779004
2.0 4399.0 160.06 560.1626160321662
2.0 5446.0 157.91 580.2159901565393
2.0 7205.0 192.81 708.4096464852333
2.0 4454.0 198.

2.0 3929.0 180.43333333333334 557.2804789531231
2.0 4530.0 151.37 584.3105108587386
2.0 3372.0 215.19666666666666 616.9519576019585
2.0 4115.0 130.56333333333333 498.04909328521245
2.0 4454.0 130.35 453.8252315961876
2.0 4493.0 183.79333333333332 620.130672215318
2.0 3308.0 161.6 539.2069794305955
2.0 6230.0 200.95666666666668 650.0271133746825
2.0 4226.0 160.72666666666666 593.2128499020305
2.0 3956.0 221.80666666666667 661.6098769583847
2.0 5748.0 200.94333333333333 725.3430453623689
2.0 4530.0 236.95333333333335 717.7086905485322
2.0 4862.0 185.39333333333335 621.3503992291485
2.0 4351.0 150.95 525.4659020018964
2.0 5004.0 219.85 650.1123499057683
2.0 4572.0 151.94666666666666 557.343679569039
2.0 4030.0 202.76333333333332 662.0801517355902
2.0 4700.0 148.97333333333333 538.0588932656185
2.0 3038.0 137.36333333333334 473.39549144686856
2.0 6305.0 142.04 569.397001280009
2.0 4172.0 114.96666666666667 407.20716131009067
2.0 3634.0 143.51666666666668 496.4648860247375
2.0 3854.0 209.27

...

2.0 3134.0 127.68666666666667 467.7774490312912
2.0 6059.0 181.23 626.0322386001113
2.0 4637.0 206.53333333333333 643.7125980504723
2.0 3934.0 182.07333333333332 630.9384502117109
2.0 4367.0 196.74333333333334 646.9825274216367
2.0 5588.0 126.18 522.0744017219512
2.0 3219.0 148.45333333333335 478.77181881235333
2.0 3934.0 159.22333333333333 576.4618230685841
2.0 4030.0 161.58666666666667 533.268602571808
2.0 3353.0 106.06 396.0254575992138
2.0 4030.0 221.94 646.3445905500668
2.0 5346.0 160.88333333333333 609.1090184760761
2.0 3734.0 170.47666666666666 502.8889036910196
2.0 3038.0 115.91 413.22997055715433
2.0 4126.0 266.23 761.4805690889295
2.0 3835.0 156.10666666666665 520.4399119548342
2.0 4030.0 203.22666666666666 602.5936789873882
2.0 4397.0 243.96666666666667 712.0873393684857
2.0 5748.0 191.26666666666668 638.6428335845388
2.0 5414.0 215.99333333333334 721.0026536859779
2.0 4678.0 190.69666666666666 575.7830534633297
2.0 7066.0 214.21 712.2822000911343
2.0 4454.0 197.663333333333

2.0 4864.0 171.41 604.4281776191444
2.0 3608.0 157.69666666666666 522.0394857245528
2.0 4454.0 154.64 571.0279243609721
2.0 5630.0 202.15333333333334 662.5552981869179
2.0 3934.0 155.95 554.4619321167745
2.0 5414.0 211.95333333333335 668.0055572290464
2.0 4057.0 165.73666666666668 622.6184925957646
2.0 5680.0 223.7 822.4139772644918
2.0 4454.0 211.49 714.7029662034431
2.0 6389.0 211.95333333333335 681.4873179222698
2.0 3437.0 104.46333333333334 427.1989723640366
2.0 6406.0 219.08333333333334 755.4052133715313
2.0 4717.0 198.57333333333332 630.286784954983
2.0 3038.0 137.31666666666666 460.37331560617434
2.0 4172.0 156.15666666666667 528.4463758246642
2.0 5286.0 200.18333333333334 651.8173029222904
2.0 4518.0 169.01333333333332 564.6891473683158
2.0 5522.0 195.47666666666666 660.6931835495875
2.0 4172.0 196.45666666666668 675.5047506289073
2.0 3306.0 171.30333333333334 513.9972548456741
2.0 4483.0 156.60666666666665 577.7312281290978
2.0 5414.0 223.53 756.2497487162117
2.0 4626.0 185.73

2.0 4746.0 122.01666666666667 515.8641452833186
2.0 4406.0 217.44666666666666 646.921474231782
2.0 4030.0 159.93 528.1514540672338
2.0 5414.0 138.75333333333333 562.5650532654473
2.0 3686.0 161.05 539.1307764973789
2.0 4768.0 171.66666666666666 607.6166682667252
2.0 3206.0 167.81333333333333 568.798545317721
2.0 3603.0 156.27333333333334 518.3268517922729
2.0 4172.0 156.84 596.8188064507798
2.0 4434.0 215.18 644.7163311100472
2.0 4859.0 168.36 567.8761517561143
2.0 5052.0 140.11666666666667 546.8717671650478
2.0 3038.0 112.96333333333334 432.599093066805
2.0 3038.0 141.44 462.7370236610278
2.0 3969.0 233.98 680.0945274494323
2.0 5303.0 257.7366666666667 767.7824348877891
2.0 5944.0 216.58 764.6976898966894
2.0 3353.0 136.28333333333333 482.3400353715439
2.0 5522.0 148.51666666666668 538.3968948544269
2.0 3038.0 204.81333333333333 567.7960301219288
2.0 5630.0 189.21333333333334 646.8884199166207
2.0 4454.0 201.04666666666665 611.581129386954
2.0 4525.0 221.98 722.2074076053223
2.0 3254.

2.0 4530.0 179.24666666666667 581.199586334639
2.0 3038.0 179.75 553.6503958576507
2.0 3280.0 196.27333333333334 626.3488686737519
2.0 3512.0 147.12333333333333 506.92802393984454
2.0 5414.0 178.82 578.1172899449846
2.0 3850.0 186.81666666666666 546.748647054161
2.0 3038.0 122.42333333333333 423.9020768867368
2.0 4172.0 153.07333333333332 522.4873344132106
2.0 4766.0 180.66666666666666 630.7967413429534
2.0 4769.0 172.99 581.0780468806809
2.0 4108.0 227.77666666666667 719.2232894742926
2.0 6024.0 134.23666666666668 567.8453140209538
2.0 5328.0 204.31666666666666 657.641115190412
2.0 4638.0 220.50666666666666 687.8812179116069
2.0 4038.0 226.84 671.1773444726314
2.0 3870.0 174.82666666666665 576.5890245997481
2.0 3530.0 141.68333333333334 498.77759544933673
2.0 3906.0 155.47 543.7529485897065
2.0 5365.0 192.69666666666666 694.2989399306965
2.0 4058.0 183.44333333333333 627.867448953643
2.0 5414.0 224.48666666666668 757.997381144699
2.0 3622.0 131.26333333333332 460.70250052380754
2.0 44

,Name,Min,Max,Calories,Gram_Prot,Gram_Fat,Gram_Carb,univariate_choice
0,Banana 1u,0,4,89,1,0,23,1
1,Mandarin 1u,0,4,40,1,0,10,6
2,Ananas 100g,0,7,50,1,0,13,3
3,Grapes 100g,0,7,76,1,0,17,3
4,Chocolate 1 bar,0,4,230,3,13,25,3


# Optimize The Shopping List Multivariately To Match Calories, Protein, Fat and Carbs

In [80]:
# in this second version, we optimize for the four components of the shopping list: calories, protein, fat and carbs
# if we need to make everything as important, we should add a weight to them
# we know that there are 30% protein calories, 20% fat and 50% carbs.
weights = (-1., -1. / 0.3, -1. / 0.2, -1./0.5)

In [81]:
creator.create("FitnessMin", base.Fitness, weights=weights)
creator.create("Individual", list, fitness=creator.FitnessMin)

C:\Users\AW639XJ\AppData\Roaming\Python\Python37\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\AW639XJ\AppData\Roaming\Python\Python37\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [82]:
def evaluate(individual):
    individual = individual[0]
    tot_prot = sum(x * y for x, y in zip(prot_data, individual))
    tot_fat = sum(x * y for x, y in zip(fat_data, individual))
    tot_carb = sum(x * y for x, y in zip(carb_data, individual))
    cals = prot_cal_p_gram * tot_prot + carb_cal_p_gram * \
        tot_carb + fat_cal_p_gram * tot_fat

    return abs(cals - total_calories), \
        abs(tot_prot - gram_prot), \
        abs(tot_fat - gram_fat), \
        abs(tot_carb - gram_carb),

In [83]:
# this is the setup of the deap library: registering the different function into the toolbox
toolbox = base.Toolbox()

toolbox.register("n_per_product", n_per_product)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.n_per_product, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [84]:
def main():
    pop = toolbox.population(n=300)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]
    
    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while g < 5000:
        # A new generation
        g = g + 1
        #print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values
            
                
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        pop[:] = offspring
        
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print(min(fits), max(fits), mean, std)
    
    
    best = pop[np.argmin([sum(toolbox.evaluate(x)) for x in pop])]
    return best

In [85]:
best_solution = main()

9.0 11521.0 3273.03 2307.8386589548813
9.0 8170.0 1724.8566666666666 1489.8716195662257
9.0 5269.0 1083.3666666666666 1063.2058967523126
9.0 6585.0 1156.2466666666667 1348.599779705685
1.0 7674.0 1035.5966666666666 1319.3568157713146
1.0 5402.0 886.7066666666667 1074.8390766166915
1.0 5472.0 721.2833333333333 993.2041564496641
1.0 7528.0 736.1966666666667 1113.836635832303
1.0 6467.0 701.49 1050.610354302044
1.0 9069.0 713.0666666666667 1124.8607745948927
0.0 6255.0 788.56 1116.2565743292773
0.0 6966.0 554.2 923.2495653938862
0.0 5654.0 669.8466666666667 1050.7542924754366
0.0 6580.0 791.7933333333333 1239.114965323593
0.0 4757.0 471.62333333333333 914.068871286817
0.0 4144.0 290.74 667.373497925512
0.0 4342.0 210.73333333333332 596.9249496842593
0.0 4471.0 261.61333333333334 675.3723544501622
0.0 2800.0 253.08333333333334 606.5913861259671
0.0 3430.0 304.5933333333333 640.4416871364809
0.0 4262.0 352.52666666666664 717.3357344755352
0.0 4954.0 305.1166666666667 741.7411923230966
0.0 3

0.0 3885.0 208.67333333333335 631.8221426600651
0.0 3860.0 156.94666666666666 585.1921654370374
0.0 4864.0 190.14666666666668 662.6892473013945
0.0 3553.0 148.68333333333334 537.9922642463262
0.0 3854.0 159.14666666666668 512.1399598634038
0.0 3553.0 136.11666666666667 465.2659272454362
0.0 4223.0 205.13333333333333 613.5138375692018
0.0 4054.0 156.35333333333332 549.8197539396181
0.0 3177.0 159.33333333333334 535.851971682064
0.0 4227.0 209.29 632.531727188447
0.0 4403.0 194.66666666666666 608.900382292174
0.0 3832.0 183.6 608.8155933176043
0.0 4480.0 155.15 523.111123471868
0.0 4975.0 225.08333333333334 708.8131886392132
0.0 4216.0 202.76 622.6284839827573
0.0 3779.0 184.76 551.6088551380105
0.0 3659.0 167.71333333333334 559.7365313153045
0.0 3548.0 172.02333333333334 553.2613151024468
0.0 4526.0 156.08666666666667 542.8744045868764
0.0 5022.0 148.81666666666666 510.23217890377015
0.0 3713.0 139.73 530.9210774054213
0.0 5593.0 208.77333333333334 667.092608730019
0.0 4535.0 209.356666

0.0 2951.0 153.38333333333333 467.5739546377188
0.0 3554.0 143.51 501.0688142028664
0.0 4616.0 184.69333333333333 608.6640994469409
0.0 2295.0 123.15 405.3978056592479
0.0 3614.0 163.86333333333334 512.6676096545293
0.0 4516.0 183.47 552.4515988754129
0.0 4373.0 144.43333333333334 533.3680894925088
0.0 4396.0 196.49 628.4329531196361
0.0 3375.0 155.09333333333333 474.0608237581146
0.0 4420.0 166.17 556.9973917652877
0.0 5321.0 193.83333333333334 645.2450017026263
0.0 4670.0 187.90333333333334 570.2801422595351
0.0 4670.0 186.04 615.6546204921501
0.0 4562.0 227.46333333333334 722.504746458842
0.0 3825.0 176.36 615.3600169006758
0.0 4166.0 196.17 613.6055419404229
0.0 4723.0 160.81333333333333 555.6944290605124
0.0 4858.0 181.60333333333332 620.2222821877832
0.0 5767.0 195.05 688.2540719094948
0.0 3249.0 128.12333333333333 418.7233471584258
0.0 4283.0 174.35 567.7756900102951
0.0 4407.0 175.05333333333334 605.6619275543815
0.0 4216.0 174.34333333333333 541.7326082015821
0.0 5107.0 200.22

0.0 4235.0 208.34 663.6571135759791
0.0 4166.0 176.80666666666667 563.6251614523807
0.0 4444.0 153.61666666666667 541.4250607322207
0.0 3642.0 124.25 470.5725670216373
0.0 5941.0 128.70666666666668 519.1445726277882
0.0 3878.0 118.56 433.4598325104646
0.0 2927.0 129.05666666666667 455.2382747406998
0.0 3644.0 155.96 505.76801506883237
0.0 4202.0 161.17 573.1335281590146
0.0 3779.0 188.14333333333335 593.2654572692471
0.0 2868.0 151.38666666666666 488.04731719600943
0.0 4561.0 141.00666666666666 543.64516609846
0.0 6044.0 218.89666666666668 701.8897676906887
0.0 3833.0 139.07 523.8897197248036
0.0 3553.0 189.68666666666667 552.5854460222016
0.0 4778.0 210.75333333333333 668.5974667083991
0.0 4778.0 182.2 579.0867580365946
0.0 4216.0 193.93 626.1032916114294
0.0 5234.0 197.14333333333335 635.1449987644991
0.0 3584.0 159.90333333333334 512.4941371913981
0.0 4541.0 128.14 470.91105359717346
0.0 3467.0 215.37 616.4699341952263
0.0 5009.0 196.55 655.5134228221417
0.0 2543.0 101.96 385.004093

0.0 4632.0 205.42333333333335 637.7060954093369
0.0 4925.0 228.60333333333332 698.5042109075332
0.0 4622.0 216.43333333333334 682.7806130489907
0.0 3734.0 185.99333333333334 555.5442496227361
0.0 4216.0 193.17666666666668 665.4527472246913
0.0 3068.0 163.38333333333333 514.638523356173
0.0 3007.0 140.81 447.18764953875905
0.0 4465.0 208.84666666666666 632.6763547203437
0.0 3697.0 169.87666666666667 544.3201889717321
0.0 3912.0 164.82333333333332 523.9853357892969
0.0 3887.0 233.85333333333332 701.1882380328093
0.0 3734.0 217.41 623.9088944442664
0.0 4360.0 226.51 656.1631071870266
0.0 4925.0 150.96333333333334 550.9052628679051
0.0 4987.0 221.54333333333332 676.841819129272
0.0 3800.0 144.46333333333334 530.1881948788457
0.0 4784.0 197.63666666666666 596.2263479492406
0.0 3960.0 177.73666666666668 556.4858434757248
0.0 5200.0 247.94666666666666 709.392052268858
0.0 3392.0 174.87666666666667 496.52309257028077
0.0 4064.0 125.58666666666667 463.3254534581736
0.0 4216.0 190.58333333333334

...

0.0 4347.0 182.21333333333334 551.521569075549
0.0 3842.0 200.49666666666667 616.1055618335402
0.0 3467.0 218.86666666666667 608.8808221282352
0.0 3371.0 164.66333333333333 522.6839357159885
0.0 2442.0 122.94666666666667 406.2378988830177
0.0 3719.0 139.26 467.6014817484364
0.0 3726.0 150.08333333333334 503.4169938750799
0.0 3248.0 179.62 546.0576608137033
0.0 3644.0 110.69333333333333 447.64420315940896
0.0 3711.0 153.66666666666666 498.1485811370829
0.0 5611.0 216.1 677.5404219183778
0.0 3761.0 138.68333333333334 472.62578895875845
0.0 3092.0 198.13666666666666 591.6282712330626
0.0 4118.0 161.14 551.1674401365402
0.0 4140.0 223.87666666666667 654.8112614503681
0.0 3276.0 155.16333333333333 537.1866869679064
0.0 4100.0 217.73666666666668 675.1365446995807
0.0 7117.0 206.95666666666668 722.3874962849848
0.0 4036.0 179.10666666666665 578.1077136851076
0.0 6212.0 166.38666666666666 557.4089974954557
0.0 4508.0 172.87666666666667 572.8512385039903
0.0 4430.0 189.61666666666667 619.552114

0.0 3227.0 171.28 530.064538460491
0.0 4094.0 176.72 611.1008331418528
0.0 4166.0 202.65 588.1610784187158
0.0 5705.0 179.71333333333334 606.5371693657547
0.0 5495.0 182.88333333333333 675.7993709099042
0.0 2585.0 135.22666666666666 433.91709878987507
0.0 4945.0 205.07 667.8850637896713
0.0 2757.0 118.35666666666667 423.6722114900727
0.0 3488.0 199.94 614.5518066580447
0.0 3152.0 154.19666666666666 463.82214765527914
0.0 6518.0 230.51 714.7406965000197
0.0 3420.0 192.68 590.8525345634052
0.0 4166.0 129.30333333333334 492.04923668493
0.0 3213.0 147.17333333333335 479.577671799771
0.0 3779.0 182.87333333333333 593.2722398884194
0.0 4216.0 231.60666666666665 661.9499618215531
0.0 4369.0 131.27333333333334 498.1659883568483
0.0 4166.0 199.69666666666666 642.3269816240186
0.0 6038.0 197.65 597.7444778777411
0.0 4549.0 168.45 581.8437598130046
0.0 3491.0 172.97666666666666 568.8228747060801
0.0 8978.0 229.87666666666667 874.6344844879806
0.0 3791.0 188.91666666666666 596.3841908162519
0.0 48

0.0 4975.0 195.68333333333334 659.2236719977691
0.0 4671.0 163.59 565.6497932172049
0.0 4723.0 159.91666666666666 550.5707187899561
0.0 5386.0 222.13333333333333 692.5401231858139
0.0 3779.0 209.45666666666668 621.9998671936264
0.0 4313.0 160.44333333333333 523.9868829041005
0.0 4880.0 176.09 550.1331613648947
0.0 4216.0 219.37333333333333 635.3821846906177
0.0 3441.0 93.10666666666667 405.51084073740316
0.0 3740.0 163.30333333333334 539.2314264972158
0.0 3554.0 200.69 587.8470043869124
0.0 4040.0 173.45333333333335 539.1289034070505
0.0 5008.0 232.86666666666667 726.7982128639435
0.0 5595.0 204.65333333333334 619.1689105746688
0.0 3392.0 109.14 429.55846369654193
0.0 3537.0 165.96333333333334 544.6539164052302
0.0 6328.0 141.56666666666666 571.4981938340273
0.0 5335.0 174.66333333333333 587.2505456125368
0.0 6103.0 154.76666666666668 593.0797857137118
0.0 3066.0 160.72666666666666 500.6904751995543
0.0 3911.0 252.70666666666668 651.4346889409218
0.0 5385.0 189.36333333333334 597.31231

0.0 3819.0 178.57 566.5038085485393
0.0 5108.0 142.74 532.655960635005
0.0 2444.0 132.19333333333333 406.5419157834637
0.0 3068.0 124.90666666666667 445.80673460842
0.0 4171.0 161.02666666666667 562.0856215520511
0.0 3482.0 204.94333333333333 582.8391946230872
0.0 4201.0 181.30333333333334 592.7975410336614
0.0 3554.0 159.65333333333334 518.8600002141447
0.0 4166.0 163.67666666666668 558.9359344226214
0.0 3248.0 166.14666666666668 549.5903976194959
0.0 4121.0 194.44333333333333 592.4704438103971
0.0 3818.0 232.63 678.7370426755858
0.0 3987.0 135.03666666666666 499.49203062266724
0.0 3002.0 157.43 486.8738355741317
0.0 4216.0 216.95333333333335 630.2299404362047
0.0 4216.0 193.04666666666665 618.7086372616938
0.0 4948.0 200.46666666666667 655.2715942433505
0.0 5186.0 201.86 683.0596462974519
0.0 3731.0 192.69 573.4635360973995
0.0 5399.0 196.65666666666667 655.1716356209433
0.0 3779.0 167.10666666666665 531.7759947780853
0.0 3986.0 141.70666666666668 519.0998304329867
0.0 5998.0 186.266

0.0 4670.0 208.84333333333333 653.9087847619795
0.0 3493.0 161.44 552.5704477560606
0.0 4085.0 167.44 524.8918933774204
0.0 3599.0 181.47666666666666 564.0225433930417
0.0 4874.0 218.64 647.6899235488125
0.0 4062.0 188.82333333333332 608.1840665365123
0.0 3275.0 221.8 632.5374191829814
0.0 3092.0 148.30333333333334 495.3467923137845
0.0 3775.0 190.72333333333333 653.3102530872211
0.0 3825.0 212.77 639.1531040629727
0.0 3554.0 224.28 648.1615808834501
0.0 7294.0 187.20333333333335 650.6504401921375
0.0 5548.0 208.93666666666667 646.2398981716378
0.0 3734.0 157.43 540.8373431695214
0.0 3878.0 186.27 545.4397404235717
0.0 4043.0 142.18333333333334 473.7710520095357
0.0 4145.0 178.60666666666665 624.8715323612971
0.0 4975.0 236.24 676.0397047511336
0.0 4238.0 192.29666666666665 637.7485047589074
0.0 4166.0 211.18 622.8286449203612
0.0 4237.0 158.26666666666668 534.1725272689422
0.0 2550.0 157.30333333333334 479.85165553764864
0.0 3214.0 160.47333333333333 518.0118685470011
0.0 3392.0 146.8

In [87]:
products_table['multivariate_choice'] = pd.Series(best_solution[0])

# Analyzing the results¶

In [88]:
products_table['univariate_gr_prot'] = products_table['univariate_choice'] * products_table['Gram_Prot']
products_table['univariate_gr_fat'] = products_table['univariate_choice'] * products_table['Gram_Fat']
products_table['univariate_gr_carb'] = products_table['univariate_choice'] * products_table['Gram_Carb']
products_table['univariate_cal'] = products_table['univariate_choice'] * products_table['Calories']

products_table['multivariate_gr_prot'] = products_table['multivariate_choice'] * products_table['Gram_Prot']
products_table['multivariate_gr_fat'] = products_table['multivariate_choice'] * products_table['Gram_Fat']
products_table['multivariate_gr_carb'] = products_table['multivariate_choice'] * products_table['Gram_Carb']
products_table['multivariate_cal'] = products_table['multivariate_choice'] * products_table['Calories']

# Making a summary of the univariate and multivariate results

In [89]:
summary = pd.DataFrame.from_records(
[
    [products_table['univariate_gr_prot'].sum(), products_table['multivariate_gr_prot'].sum(), gram_prot],
    [products_table['univariate_gr_fat'].sum(), products_table['multivariate_gr_fat'].sum(), gram_fat],
    [products_table['univariate_gr_carb'].sum(), products_table['multivariate_gr_carb'].sum(), gram_carb],
    [products_table['univariate_cal'].sum(), products_table['multivariate_cal'].sum(), sum((cal_prot, cal_carb, cal_fat))]
])
summary.columns = ['univariate', 'multivariate', 'goal']
summary.index = ['prot', 'fat', 'carb', 'cal']
summary["univ_error"] = (summary["goal"] - summary["univariate"]).apply(abs)
summary["multiv_error"] = (summary["goal"] - summary["multivariate"]).apply(abs)
summary

,univariate,multivariate,goal,univ_error,multiv_error
prot,817,860,1312.500000,495.500000,452.500000
fat,1018,1040,388.888889,629.111111,651.111111
carb,1267,1175,2187.500000,920.500000,1012.500000
cal,17779,17646,17500.000000,279.000000,146.000000


In [90]:
summary["univ_error"].sum(), summary["multiv_error"].sum()

(2324.1111111111113, 2262.1111111111113)

In [91]:
# Shopping list
products_table[['Name', 'multivariate_choice', 'univariate_choice']]

,Name,multivariate_choice,univariate_choice
0,Banana 1u,8,1
1,Mandarin 1u,6,6
2,Ananas 100g,5,3
3,Grapes 100g,2,3
4,Chocolate 1 bar,8,3
5,Hard Cheese 100g,2,4
6,Soft Cheese 100g,1,4
7,Pesto 100g,8,2
8,Hoummous 100g,6,8
9,Aubergine Paste 100g,3,3
